## install packages

In [1]:
pip install transformers torch scikit-learn datasets

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached pyarrow-18.1.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached multidict-6.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.0 kB)
  Using cached propcache-0.2.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached yarl-1.18.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (69 kB)
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.0/456.0 kB 12.9 MB/s

## 1.loading the cnn daily mail dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

print(train_data[0]["article"])

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

## 2. Preprocessing the data

In [18]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

def preprocess_function(examples):
    inputs = tokenizer(
        examples["article"],
        max_length = 1024,
        truncation = True,
        padding = "max_length"
    )
    targets = tokenizer(
        examples["highlights"],
        max_length = 128,
        truncation = True,
        padding = "max_length",
    )
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_train_data = train_data.map(preprocess_function, batched = True)
tokenized_validation_data = validation_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

## 2. noising and text corruption

In [22]:
import random

def apply_noise(text, mask_prob=0.15):
    words = text.split()
    num_words = len(words)
    num_masked = int(mask_prob * num_words)
    masked_text = []
    for word in words:
        if random.random() < mask_prob:
            masked_text.append("[MASK]")
        else:
            masked_text.append(word)
    return " ".join(masked_text)

temp_data = [apply_noise(article) for article in train_data['article']]

train_data = train_data.add_column("noised_data", temp_data)

print("original:", train_data[0]["article"])
print("Noised:", train_data[0]["noised_data"])


original: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Deta

## 3. Dual-stream Encoding

In [4]:
from transformers import BartForConditionalGeneration, BartTokenizer

semantic_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
syntactic_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

## 4. Hierarchical Attention

In [5]:
from transformers import BertTokenizer, BertModel

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_sentence_embedding(text):
    inputs = bert_tokenizer(text, return_tensors='pt', padding = True, truncation=True)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

embedding = get_sentence_embedding("This is an example sentence.")
print("Sentence Embedding:", embedding.shape)

Sentence Embedding: torch.Size([1, 768])


## 5. Sparse Attention for long Documents

In [6]:
from transformers import LongformerTokenizer, LongformerForSequenceClassification

longformer_tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
longformer_model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096")

long_text = " ".join([train_data[i]['article'] for i in range(5)])
inputs = longformer_tokenizer(long_text, return_tensors="pt", max_length=4096, truncation=True)
outputs = longformer_model(**inputs)

print("Longformer Outputs Shape:", outputs.logits.shape)


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Initializing global attention on CLS token...
Input ids are automatically padded to be a multiple of `config.attention_window`: 512


Longformer Outputs Shape: torch.Size([1, 2])


## 6. Knowledge graph integration

In [7]:
def get_entity_embedding(entity):
    return [0.0] * 768

entity_embedding = get_entity_embedding("Barack Obama")
print("Entity Embedding:", entity_embedding)

Entity Embedding: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

## 7. Fusion Decoding

In [12]:
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

def generate_summary(input_text, max_length=150, min_length=50):
    inputs = tokenizer(input_text, return_tensors = "pt", max_length = 1024, truncation = True)
    summary_ids = bart_model.generate( 
                    inputs['input_ids'],
                    max_length = max_length,
                    min_length = min_length,
                    num_beams = 5, 
                    do_sample = True,
                    top_p = 0.9,
                    no_repeat_ngram_size = 2)
    return tokenizer.decode(summary_ids[0], skip_special_tokens = True)

article = train_data[1]['article']
print("before:", article)
print("length of the article:", len(article))
summary = generate_summary(article, max_length=600, min_length=50)
print("Generate Summary:", summary)

before: Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to fo

## 8. Evaluate and Compare

In [14]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.9 MB/s eta 0:00:00a 0:00:01
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9a7c7e0f80e1cc2a65435c47d688f7f42c7f16619f53260d00e98ecb99729df4
  Stored in directory: /Users/hari/Library/Caches/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from rouge_score import rouge_scorer

# Reference summary and generated summaries
reference_summary = train_data[0]['highlights']
bart_summary = generate_summary(train_data[0]['article'])

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, bart_summary)

print("ROUGE Scores:", scores)

NameError: name 'train_data' is not defined